# Factual Consistency

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from openai import OpenAI

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/DISSERTATION/Dataset/Extracted Dataset (csv)/test_plos.csv')

## Pegasus

In [ ]:
peg_df=pd.read_csv('/content/drive/MyDrive/DISSERTATION/Dataset/test_peg_v4.csv')

In [ ]:
#Replacing the abstact without stopwords to the original abstract for evaluation
peg_df = peg_df.drop(columns=["abstract"])
peg_df["abstract"] = test_df["abstract"]

### Summary Ranking

In [ ]:
df=peg_df.copy()

In [ ]:
#Summary Ranking Prompt
df['Message']="Decide which of the following summary is more consistent with the article. Note that consistency means all information in the summary is supported by the article.\nArticle: "+df['abstract']+"\nSummary A: "+df['plain language summary']+"\nSummary B: "+df['generated_summary']+"\nExplain your reasoning step by step then answer (A or B) the question: "

In [ ]:
#API Key
client = OpenAI(api_key="PASTE YOUR API KEY HERE")

In [ ]:
#GPT response generation
def get_response(message):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        stream=True,
    )
    response_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content
    return response_content

In [ ]:
df["Response"]=df["Message"].apply(get_response)

In [ ]:
df.to_csv('/content/drive/MyDrive/DISSERTATION/Results/Factual Consistency/peg_v4_summaryRanking.csv', index=False)

### Consistency Rating

In [ ]:
df_copy=peg_df.copy()

In [ ]:
#Consistency Rating Prompt
df_copy['Message']="Score the following summary given the corresponding article with respect to consistency from 1 to 10. Note that consistency measures how much information included in the summary is present in the source article. 10 points indicate the summary contains only statements that are entailed by the source document.\nSummary: " + df_copy["generated_summary"] + "\nSource Article: " + df_copy["abstract"] + "\nMarks: "

In [ ]:
print(df_copy['Message'][0])

Score the following summary given the corresponding article with respect to consistency from 1 to 10. Note that consistency measures how much information included in the summary is present in the source article. 10 points indicate the summary contains only statements that are entailed by the source document.
Summary: Epigenetics is the study of changes in gene expression that occur during the initiation and progression of cancer. However, it is not well understood how epigenetic changes occur in the early stages of cancer development. Here, we show that a large set of differentially methylated regions (DMRs) are present in the intestinal adenoma of a mouse model of colon cancer. These DMRs are highly enriched in DNA methyltransferases-regulated adenoma cells. We also show that these DMRs are frequently found in colon cancer, rare in adenoma. These data allow the distinction between early and late epigenetic alterations occurring in intestinal adenoma, and facilitate selection of specif

In [ ]:
#API Key
client = OpenAI(api_key="PASTE YOUR API KEY HERE")

In [ ]:
#GPT response generation
def get_response(message):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        stream=True,
    )
    response_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content
    return response_content

In [ ]:
df_copy["Response"]=df_copy["Message"].apply(get_response)

In [ ]:
df_copy.to_csv('/content/drive/MyDrive/DISSERTATION/Results/Factual Consistency/peg_v4_consistencyRating2.csv', index=False)

## BART

In [ ]:
bart_df=pd.read_csv('/content/drive/MyDrive/DISSERTATION/Dataset/test_bart_v2.csv')

In [ ]:
bart_df = bart_df.drop(columns=["abstract"])
bart_df["abstract"] = test_df["abstract"]

### Summary Ranking

In [ ]:
bart_df

,plain language summary,generated_summary,abstract
0,The formation and progression of tumours to me...,Aberrant CpG methylation is a universal epigen...,Aberrant CpG methylation is a universal epigen...
1,Chagas disease remains a major public health p...,Trypanosoma cruzi strains are currently classi...,Trypanosoma cruzi strains are currently classi...
2,The work described here fills a current gap in...,DNA polymerase ν (Pol ν) is a family of DNA po...,"DNA polymerase ν (pol ν), encoded by the POLN ..."
3,Vaccines are considered a cost-effective way t...,Dengue virus is a mosquito-transmitted virus t...,Dengue virus is a mosquito-transmitted virus t...
4,Cells sense their surroundings and respond to ...,Cells reliably sense environmental changes des...,Cells reliably sense environmental changes des...
...,...,...,...
995,The demographic history of our species is refl...,Genetic models built from genetic data play im...,Demographic models built from genetic data pla...
996,Hair plays an important role in primates and i...,Hair plays an important role in primates and i...,Hair plays an important role in primates and i...
997,The human whipworm Trichuris trichiura is toge...,Trichuris trichiura infections are the most co...,The single-dose benzimidazoles used against Tr...
998,Clinical management of upper and lower respira...,"In order to suppress microbial outgrowth, the ...","In an effort to suppress microbial outgrowth, ..."


In [ ]:
df=bart_df.copy()

In [ ]:
#Summary Ranking API
df['Message']="Decide which of the following summary is more consistent with the article. Note that consistency means all information in the summary is supported by the article.\nArticle: "+df['abstract']+"\nSummary A: "+df['plain language summary']+"\nSummary B: "+df['generated_summary']+"\nExplain your reasoning step by step then answer (A or B) the question: "

In [ ]:
#API Key
client = OpenAI(api_key="PASTE YOUR API KEY HERE")

In [ ]:
#API Response Generation
def get_response(message):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        stream=True,
    )
    response_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content
    return response_content

In [ ]:
df["Response"]=df["Message"].apply(get_response)

In [ ]:
df.to_csv('/content/drive/MyDrive/DISSERTATION/Results/Factual Consistency/bart_v2_summaryRanking.csv', index=False)

### Consistency Rating

In [ ]:
df=bart_df.copy()

In [ ]:
#Consistency Rating Prompt
df['Message']="Score the following summary given the corresponding article with respect to consistency from 1 to 10. Note that consistency measures how much information included in the summary is present in the source article. 10 points indicate the summary contains only statements that are entailed by the source document.\nSummary: " + df["generated_summary"] + "\nSource Article: " + df["abstract"] + "\nMarks: "

In [ ]:
print(df['Message'][0])

Score the following summary given the corresponding article with respect to consistency from 1 to 10. Note that consistency measures how much information included in the summary is present in the source article. 10 points indicate the summary contains only statements that are entailed by the source document.
Summary: Aberrant CpG methylation is a universal epigenetic trait in cancer cell genomes. However, human cancer samples from cell lines preclude the investigation of epigenetic changes occurring during early tumour development. Here, we present a list of 13,000 recurring differentially methylated regions (DMRs) characterizing the intestinal adenoma mouse. Polycomb Repressive Complex (PRC) targets strongly enriched hypermethylated DMRs, PRC2 components of DNA methyltransferases-regulated adenomas. We demonstrate that the DMR signature arises from de novo adenomatous cells and expansion of the pre-existing pattern of intestinal stem cells and undifferentiated crypt cells. We found th

In [ ]:
#API Key
client = OpenAI(api_key="PASTE YOUR API KEY HERE")

In [ ]:
#API Response Generation
def get_response(message):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        stream=True,
    )
    response_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content
    return response_content

In [ ]:
df["Response"]=df["Message"].apply(get_response)

In [ ]:
df.to_csv('/content/drive/MyDrive/DISSERTATION/Results/Factual Consistency/bart_v2_consistencyRating2.csv', index=False)